# SymSpell

In [1]:
import pandas as pd
from os import path
from symspellpy import SymSpell, Verbosity
from symspellpy.helpers import DictIO

In [2]:
# csv to list
data = pd.read_csv('dataset_word.csv')

val_list = data.values.tolist()

train_words = []
for word in val_list :
    train_words.append(word[0])

# list to dictionary
word_dict = {word: 1 for word in train_words}

In [3]:
sym_spell = SymSpell(max_dictionary_edit_distance=6, prefix_length=7)
dict_stream = DictIO(word_dict)             # SymSpell용으로 딕셔너리 형태 변환

if not path.exists("symspell.pickle") : 
    sym_spell.load_dictionary_stream(dict_stream, 0, 1)
    sym_spell.save_pickle('symspell.pickle')
    print("pickle created!")
else : 
    sym_spell.load_pickle('symspell.pickle')

In [4]:
input_term = input()

suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=6)
for suggestion in suggestions:
    print(suggestion)

크동
동, 1, 1
크롬, 1, 1


In [5]:
suggestion_list = []
for suggestion in suggestions :
    suggestion_list.append(suggestion.term)

In [6]:
suggestion_list

['동', '크롬']

# 자음모음 분리
- 입력과 후보 단어의 길이 같을 때만 판단 가능

In [7]:
from jamo import h2j, j2hcj

In [8]:
# 자음 모음 받침 나누고 리스트로
j_list = []

if len(suggestion_list) == 1:
    print(suggestion_list[0])

else:
    cnt_suggestion = len(suggestion_list)
    for i in range(cnt_suggestion):
        for j in range(len(suggestion_list[i])):
            j_data= j2hcj(h2j(suggestion_list[i][j]))
            j_list.append(j_data)
    
print(j_list)
print(len(j_list[0]))

['ㄷㅗㅇ', 'ㅋㅡ', 'ㄹㅗㅁ']
3


In [9]:
# input 단어와 비교

j_input_list = []
hamming_val = 0
hamming_list = []


for i in range(len(input_term)):
    j_input = j2hcj(h2j(input_term[i]))
    j_input_list.append(j_input)

cnt_j_list = len(j_list)

check = 0
tmp = 0
s = 0


i = 0
while(True):
    if i == cnt_j_list:
        break
        
    input_sylla = j_input_list[tmp]
    suggest_sylla = j_list[i]
    
    if len(input_sylla) + len(suggest_sylla) < 6 :
        r = 2
    else:
        r = 3
        
        
    for j in range(r):
        input_phone = j_input_list[tmp][j]
        suggest_phone = j_list[i][j]
        print("input_phone:",input_phone, "suggest_phone:", suggest_phone)
        
        if input_phone == suggest_phone:
            hamming_val -= 1
        else:
            hamming_val += 1
            
    check += 1
    tmp += 1
    i += 1
    
    print(tmp, len(suggestion_list[s]))
    
    if check == len(suggestion_list[s]) or tmp == len(input_term):  # 초기화 기준이 input단어 길이임
        
        if len(input_term) < len(suggestion_list[s]):
            i+= len(suggestion_list[s]) - len(input_term)
        hamming_list.append(hamming_val)
        hamming_val = 0
        check = 0
        tmp = 0
        s += 1
        print("초기화")
        
    if(s+1 > len(suggestion_list)) :
        print("all checked")
        break

print(hamming_list)

input_phone: ㅋ suggest_phone: ㄷ
input_phone: ㅡ suggest_phone: ㅗ
1 1
초기화
input_phone: ㅋ suggest_phone: ㅋ
input_phone: ㅡ suggest_phone: ㅡ
1 2
input_phone: ㄷ suggest_phone: ㄹ
input_phone: ㅗ suggest_phone: ㅗ
input_phone: ㅇ suggest_phone: ㅁ
2 2
초기화
all checked
[2, -1]


In [10]:
print(j_input_list)
print(j_list)

['ㅋㅡ', 'ㄷㅗㅇ']
['ㄷㅗㅇ', 'ㅋㅡ', 'ㄹㅗㅁ']


In [11]:
count_fmhd = len(hamming_list)
min_2 = 100
check_i = 0
for i in range(count_fmhd):
    if hamming_list[i] < min_2:
        min_2 = hamming_list[i]
        check_i = i
print(hamming_list)
print(suggestion_list[check_i])

[2, -1]
크롬
